### Steps to get similar/duplicate questions

1. Preprocess data - clean question title and body.
remove html, punctuations, stopwords, words less than 4 chars, but need to keep words like SQL, C++, C#...

2. Apply TF-IDF count vectorizer and create sparse matrix.

3. Use NearestNeighbors and get most similar/duplicate 5 questions.

Challenges - keeping words like SQL, C#, C++ ....

For nearest neighbors, change metrics (mostly need to use cosine metric)

How to relate title and body ??

Can we use title , body and tag together to identify similar questions? Are tags reliable?

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from string import punctuation
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from nltk.tokenize import ToktokTokenizer

In [18]:
questionsDF = pd.read_csv("Questions.csv", encoding='ISO-8859-1')
answersDF = pd.read_csv("Answers.csv", encoding='ISO-8859-1')
tagsDF = pd.read_csv("Tags.csv", encoding='ISO-8859-1')

In [3]:
#questionsDF = questionsDF.head(1000)

In [5]:
#test = questionsDF['Body'].head(1).apply(lambda x: BeautifulSoup(x).get_text()) 

In [ ]:
#pd.value_counts(tagsDF['Tag'].head(1000)).plot(kind='bar')

In [7]:
#pd.Series(' '.join(tagsDF['Tag']).split())

In [9]:
tagsDF['Tag'] = tagsDF['Tag'].astype(str)
grouped_tags = tagsDF.groupby("Id")['Tag'].apply(lambda tags: ' '.join(tags))
grouped_tags.reset_index()
grouped_tags_final = pd.DataFrame({'Id':grouped_tags.index, 'Tags':grouped_tags.values})
#questionsDF.drop(columns=['OwnerUserId', 'CreationDate', 'ClosedDate'], inplace=True)
df = questionsDF.merge(grouped_tags_final, on='Id')
new_df = df[df['Score']>5]
new_df.drop(columns=['Id', 'Score'], inplace=True)
new_df['Tags'] = new_df['Tags'].apply(lambda x: x.split())


# new_df = tagsDF


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [14]:
flat_list = [item for sublist in new_df['Tags'].values for item in sublist]

In [35]:
test = questionsDF['Body']
#test = questionsDF['Body'].head(10000).apply(lambda x: BeautifulSoup(x).get_text()) 

In [36]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub(r"\'\n", " ", text)
    text = re.sub(r"\'\xa0", " ", text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

test = test.apply(lambda x: clean_text(x)) 
token = ToktokTokenizer()
punct = '!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def clean_punct(text): 
    words=token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    remove_punctuation = str.maketrans(' ', ' ', punct)
    for w in words:
        if w in flat_list:
            punctuation_filtered.append(w)
        else:
            punctuation_filtered.append(regex.sub('', w))
  
    filtered_list = strip_list_noempty(punctuation_filtered)
        
    return ' '.join(map(str, filtered_list))

test = test.apply(lambda x: clean_punct(x)) 

In [42]:
x_train = test.str.join(' ')
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(test)


In [61]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(n_neighbors=5,metric="cosine").fit(x_train)
distances, indices = model_knn.kneighbors(x_train)

In [62]:
indices, distances

(array([[   0, 7678, 3782, 5444, 4659],
        [   1,  748, 1682,  394,  255],
        [   2, 1238, 2450,  201, 2005],
        ...,
        [9997, 7880,  173, 2041, 4309],
        [9998, 9422, 8281, 3991, 8634],
        [9999, 5234, 3649, 6205, 8961]]),
 array([[0.        , 0.76595854, 0.77973442, 0.79624491, 0.8076016 ],
        [0.        , 0.78596133, 0.83338084, 0.84229343, 0.84646255],
        [0.        , 0.71826958, 0.75315654, 0.76627653, 0.77252958],
        ...,
        [0.        , 0.78609394, 0.80441507, 0.81281784, 0.82508205],
        [0.        , 0.60285395, 0.73795839, 0.78185969, 0.80519574],
        [0.        , 0.71855159, 0.78059459, 0.78630231, 0.81248861]]))

In [63]:
test[7678]

'i am having a brain-dead moment i have two tables described by create table tablea id integer primary key autoincrement name varchar 255 not null unique name create table tableb id integer primary key autoincrement akey integer not null otherstuff varchar 255 not null foreign key akey references tablea id on delete cascade how can i select all rows from tablea that do not have an entry in tablebakey'

In [64]:
test[3782]

'i want to create a history table to track field changes across a number of tables in db2 i know history is usually done with copying an entire table structure and giving it a suffixed name eg user -- userhistory then you can use a pretty simple trigger to copy the old record into the history table on an update however for my application this would use too much space it does not seem like a good idea to me at least to copy an entire record to another table every time a field changes so i thought i could have a generic history table which would track individual field changes create table history historyid long generated always as identity recordid integer not null tablename varchar 32 not null fieldname varchar 64 not null fieldvalue varchar 1024 changetime timestamp primary key historyid ok so every table that i want to track has a single auto-generated id field as the primary key which would be put into the are cordid field and the maximum varchar size in the tables is 1024 obviously 

In [57]:
test[5444]

'i have built a database in ms access there i have a table called customers which also has a cell called employee type integer i also built a program in c++ which controls all data let say i have a string like this string sqlstring select from customers where customersemployee id id passes through my function correctly and is an integer so i get an error in compilation saying invalid pointer addition if i declare id as a string of course there no error but there are no results in my form also if i declare in database cell employee as text and build my query like this string sqlstring select from customers where customersemployee 128 i get results but i need that employee as an integer cause its a foreign key from another table so what should i do with my query to have results passing integer as parameter through variable id to be ok with the cell employee from database which is also integer any ideas i would really appreciate some help here as i said if i convert id to string there are

In [58]:
test[4659]

'is there a measurable performance difference between using int vs varchar as a primary key in mysql i would like to use varchar as the primary key for reference lists think us states country codes and a coworker wo not budge on the int autoincrement as a primary key for all tables my argument as detailed here is that the performance difference between int and varchar is negligible since every int foreign key reference will require a join to make sense of the reference a varchar key will directly present the information so does anyone have experience with this particular use-case and the performance concerns associated with it'

In [ ]:
#train = questionsDF["Body"]

In [ ]:
# from IPython.display import Audio
# sound_file = "bottle_pop_2.wav"
# Audio(filename=sound_file, autoplay=True)

In [ ]:
# def remove_punctuation(words):
#     """Remove punctuation from list of tokenized words"""
    
#     new_words = []
#     for word in words:
#         new_word = re.sub(r'[^\w\s]', '', word)
#         if new_word != '':
#             new_words.append(new_word)
#     return new_words



# def lemmatize(words):
#     text = " ".join(WordNetLemmatizer().lemmatize(word) for word in words.split())
#     return text.split()

# def remove_less4(words):
#     text = re.sub(r'\b\w{1,3}\b', '', words)
#     return text

# def preprocess_data(text):  
#     text = text.lower()
#     text = re.sub(r'\b\w{1,3}\b', '', text)   #remove words less than 3
#     text = re.sub(r'[^\w\s]','',text)
#     text = re.sub(r'\d+', '', text)
#     return text





# train = train.apply(lambda x: " ".join(x.lower() for x in x.split()))
# train = train.str.replace('[^\w\s]','')
# stop = stopwords.words('english')
# train = train.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
# freq = pd.Series(' '.join(train[1]).split()).value_counts()[:10]
# freq = list(freq.index)
# train[1] = train[1].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# freq = pd.Series(' '.join(train[1]).split()).value_counts()[-10:]
# freq = list(freq.index)
# train[1] = train[1].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# from textblob import TextBlob
# TextBlob(train[1][1]).words
# from textblob import Word
# train[1] = train[1].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))


# x_train = train[1]
# #x_train = train[1].apply(preprocess_data)
# #train[1] = train[1].apply(lambda x: " ".join(x.lower() for x in x.split()))
# #freq = pd.Series(' '.join(x_train).split()).value_counts()[:30]
# #x_train = x_train.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
# # from IPython.display import Audio
# # sound_file = "bottle_pop_2.wav"
# # Audio(filename=sound_file, autoplay=True)
# # x_train_unVectorized = x_train
# #x_train = x_train.str.join(' ')

In [ ]:
#questionsDF.loc[questionsDF['Id'] == 80]
#answersDF.loc[answersDF['ParentId'] == 80]
import pickle
filename = 'similar_questions_model'
outfile = open(filename,'wb')
pickle.dump(similar_questions_model,outfile)
outfile.close()